In [1]:
import pandas as pd
import os
import csv
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor

os.chdir('Resources/')

import warnings
warnings.filterwarnings("ignore")

In [2]:
def reset_seeds(seed=42):
    import os
    import random
    import numpy as np
    import tensorflow as tf
    import torch
    import torch.backends.cudnn as cudnn

    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    torch.manual_seed(seed)
    cudnn.deterministic = True
    cudnn.benchmark = False
    
def preprocess_dataset(df, seed=42):
    reset_seeds()

    df.rename(columns={
        "Store": "Plant_ID",
        "Dept": "Machine_Type",
        "Weekly_Sales": "Weekly_Production",
        "IsHoliday": "Quality_Audit",
        "Year": "Year",
        "Month": "Month",
        "Week": "Week"
    }, inplace=True)

    X = df[['Plant_ID', 'Machine_Type', 'Quality_Audit', 'Year', 'Month', 'Week']].values
    y = df['Weekly_Production'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
    
    mlp_model = MLPRegressor(
        hidden_layer_sizes=(100, 50),
        activation='relu',
        solver='adam',
        max_iter=50,
        random_state=42
    )

    mlp_model.fit(X_train, y_train)
    predicted_sales = mlp_model.predict(X)

    columns = ['Plant_ID', 'Machine_Type', 'Quality_Audit', 'Year', 'Month', 'Week']

    df = pd.DataFrame(X, columns=columns)
    df["Weekly_Production"] = predicted_sales.astype(int)

    return df

In [3]:
clients = list(range(1, 10))

for client in clients:
    df = pd.read_csv(f"1_Client_{client}_Data.csv")
    df = preprocess_dataset(df) 

    grouped = df.groupby(['Year', 'Month'])

    for (year, month), group in grouped:
        filename = f"2_{client}_Client_Data_{year}_{month}.csv"
        group.to_csv(filename, index=False)